이 코드는 Yahoo!의 뉴스 추천 시스템 로그 데이터를 읽고 전처리하는 함수들로 구성. 전체적으로 "사용자가 뉴스 기사 중 하나를 클릭했는지 여부"를 다루는 **contextual bandit** 형식의 데이터

---

### 데이터 한 줄 예시

```
1241160900 109513 0 |user 2:0.000012 3:0.000000 4:0.000006 5:0.000023 6:0.999958 1:1.000000 |109498 2:0.306008 3:0.000450 4:0.077048 5:0.230439 6:0.386055 1:1.000000 |109509 2:0.306008 3:0.000450 4:0.077048 5:0.230439 6:0.386055 1:1.000000 ...
```

### 🔹 구성 요소

1. `1241160900`: 타임스탬프 (사용 시각)
2. `109513`: 사용자가 클릭한 기사 ID (displayed article)
3. `0`: 클릭 여부 (`1`: 클릭함, `0`: 클릭 안 함)
4. `|user ...`: 사용자 피처 (6개)
5. `|109498 ...`: 추천된 기사 ID 및 해당 기사에 대한 6개의 피처
6. `...`: 기사 여러 개가 추천될 수 있음 (pool로 구성됨)

---

## 🔸 코드 구조 설명

### 1. `get_yahoo_events(filenames)`

Yahoo 이벤트 로그 파일을 읽어서 다음과 같은 변수들을 생성:

#### 생성되는 전역 변수

* `articles`: 전체 기사 ID 목록 (`[str]`)
* `features`: 각 기사에 대한 feature 벡터 (`np.array([[f1, f2, ..., f6], ...])`)
* `events`: 개별 사용자 이벤트 로그 (`[ [displayed_index, click, user_features, pool_indexes] ]`)
* `n_arms`: 사용된 기사 수
* `n_events`: 전체 이벤트 수

#### 📌 주요 처리

```python
if (len(cols) - 10) % 7 != 0:
```

* `10`은 고정된 첫 10개의 열 (timestamp, clicked id, click 여부, user features 등)
* 이후는 기사 정보가 7개 단위로 구성됨 (`article_id + 6 features`)
* 정상적인 데이터인지 확인하고 아니라면 `skipped`로 넘김

---

### 2. `max_articles(n_articles)`

기사 수를 최대 `n_articles`개로 제한합니다.

#### * 처리 방식

* `articles`, `features`를 자릅니다: `[:n_articles]`
* `events`도 필터링:

  * 만약 사용자가 클릭한 기사 index가 잘린 이후에 있다면 → 그 이벤트 제거
  * 남은 이벤트는 모든 기사를 다시 pool로 설정 (`np.arange(0, n_arms)`)

---

## 🔸 예시: 한 이벤트의 의미

```
[
  0,                          # 사용자가 클릭한 기사 (pool 내 index)
  1,                          # 클릭했는지 여부
  [0.000012, ..., 1.0],       # 사용자 피처
  [12, 45, 78, 103, 109]      # pool 안의 기사 index (전체 article 리스트 기준)
]
```

이 데이터는 **contextual bandit** 문제로:

* **context**: 사용자 피처 + 기사 피처
* **action**: 추천한 기사
* **reward**: 클릭 여부

---

## 🔸 요약

| 요소                   | 설명                             |
| -------------------- | ------------------------------ |
| `articles`           | 고유한 기사 ID 리스트                  |
| `features`           | 기사별 피처 (6차원 벡터)                |
| `events`             | 사용자 행동: 어떤 기사 pool에서 무엇을 클릭했는가 |
| `get_yahoo_events()` | 로그 파일을 읽고 전처리                  |
| `max_articles(n)`    | 기사 수를 n개로 제한하고 events도 필터링     |
